In [14]:
import pandas as pd

In [16]:
df=pd.read_csv('../artifacts/glassdoor_jobs.csv')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN
1,Python Data Scientist/Analyst,₹31L,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN
2,"Lead Data Scientist, India - BCG X",₹8L,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD)
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,51 to 200 Employees,2006,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,51 to 200 Employees,2007,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)


In [19]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [22]:
df['Salary Estimate'] = df['Salary Estimate'].str.replace('₹', ' ')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('L', ' ')

In [23]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN
1,Python Data Scientist/Analyst,31,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN
2,"Lead Data Scientist, India - BCG X",8,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD)
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,51 to 200 Employees,2006,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,51 to 200 Employees,2007,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)


In [24]:
#parsing 'Job Description'
skills = {
    'python': 'python_yn',
    'sql': 'sql_yn',
    'excel': 'excel_yn',
    'spark': 'spark_yn',
    'aws': 'aws_yn'
}
# Loop through the skills dictionary to create and populate the columns
for skill, column in skills.items():
    df[column] = df['Job Description'].str.contains(skill, case=False, na=False).astype(int)

In [25]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,python_yn,sql_yn,excel_yn,spark_yn,aws_yn
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0
1,Python Data Scientist/Analyst,31,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0
2,"Lead Data Scientist, India - BCG X",8,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD),0,0,0,1,0
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,51 to 200 Employees,2006,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,0,0,0
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,51 to 200 Employees,2007,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,0,0,0


In [27]:
df['Size']=df['Size'].str.replace('Employees',' ')
df['Size']=df['Size'].str.replace('+',' ')
df['Size']=df['Size'].str.replace('to','-')

In [31]:
# Extract the upper limit of the size range
df['Size_Upper'] = df['Size'].apply(lambda x: x.split('-')[1].strip() if '-' in str(x) else x)

In [32]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN
1,Python Data Scientist/Analyst,31,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN
2,"Lead Data Scientist, India - BCG X",8,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD),0,0,0,1,0,10000
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,51 - 200,2006,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,0,0,0,200
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,51 - 200,2007,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,0,0,0,200


In [36]:
df['Founded'].value_counts()

Founded
--      166
1963     68
2006     67
2007     67
1995     33
1967     33
1851     33
2009     33
2018     33
2019     33
1889     33
1998     33
2015     33
1799     33
1972     33
Name: count, dtype: int64

In [37]:
# Convert the 'Founded' column to numeric, forcing errors(i.e '--' values) to NaN
df['Founded'] = pd.to_numeric(df['Founded'], errors='coerce')

In [38]:
df['Age'] = 2024 - df['Founded'].dropna().astype(int) # considering nan values

In [39]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper,Age
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN,NaN
1,Python Data Scientist/Analyst,31,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN
2,"Lead Data Scientist, India - BCG X",8,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000,1963.0,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD),0,0,0,1,0,10000,61.0
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,51 - 200,2006.0,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,0,0,0,200,18.0
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,51 - 200,2007.0,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,0,0,0,200,17.0


In [40]:
df['Type of ownership'].value_counts()

Type of ownership
Company - Private                 401
Company - Public                  198
Subsidiary or Business Segment     66
Unknown                            33
Private Practice / Firm            33
Name: count, dtype: int64

In [41]:
df['Industry'].value_counts()

Industry
Information Technology Support Services    135
--                                         132
Business consulting                         68
Internet & Web Services                     66
Telecommunications Services                 33
Electronics Manufacturing                   33
Crop Production                             33
Software Development                        33
Computer Hardware Development               33
Machinery Manufacturing                     33
HR Consulting                               33
Enterprise Software & Network Solutions     33
Banking & Lending                           33
Advertising & Public Relations              33
Name: count, dtype: int64

In [42]:
df['Industry'] = df['Industry'].apply(lambda x: 'unknown' if x == '--' else x)

In [45]:
df['Sector'].value_counts()

Sector
Information Technology          300
--                              132
Management and consulting        68
Manufacturing                    66
Telecommunications               33
Agriculture                      33
Human resources and staffing     33
Finance                          33
Media and communication          33
Name: count, dtype: int64

In [46]:
df['Sector'] = df['Sector'].apply(lambda x: 'unknown' if x == '--' else x)

In [48]:
df['Revenue'].value_counts()

Revenue
Unknown / Non-Applicable      331
$10+ billion (USD)            198
$5 to $25 million (USD)       101
$5 to $10 billion (USD)        68
$100 to $500 million (USD)     33
Name: count, dtype: int64

In [69]:
df['Revenue']=df['Revenue'].str.strip()
df['Revenue']=df['Revenue'].str.replace('billion',',000')
df['Revenue']=df['Revenue'].str.replace('million',' ')
df['Revenue']=df['Revenue'].str.replace('$',' ')
df['Revenue']=df['Revenue'].str.replace('to','-')
df['Revenue']=df['Revenue'].str.replace('Unknown / Non-Applicable','unknown')
df['Revenue']=df['Revenue'].str.replace('USD',' ')
df['Revenue']=df['Revenue'].str.replace('(',' ')
df['Revenue']=df['Revenue'].str.replace(')',' ')

In [71]:
df['Revenue_Upper'] = df['Revenue'].apply(lambda x: x.split('-')[1].strip() if '-' in str(x) else x)

In [72]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper,Age,Revenue_Upper
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN,NaN,NaN
1,Python Data Scientist/Analyst,31,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN,NaN
2,"Lead Data Scientist, India - BCG X",8,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000,1963.0,Company - Private,Business consulting,Management and consulting,"5 - 10 ,000",0,0,0,1,0,10000,61.0,"10 ,000"
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,51 - 200,2006.0,Company - Private,Information Technology Support Services,Information Technology,5 - 25,0,0,0,0,0,200,18.0,25
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,51 - 200,2007.0,Company - Private,Information Technology Support Services,Information Technology,5 - 25,0,0,0,0,0,200,17.0,25


In [73]:
df = df.drop(['Size', 'Founded', 'Revenue'], axis=1)

In [74]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Type of ownership,Industry,Sector,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper,Age,Revenue_Upper
0,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,1,0,1,0,0,NaN,NaN,NaN
1,Python Data Scientist/Analyst,31,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN,NaN
2,"Lead Data Scientist, India - BCG X",8,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,Company - Private,Business consulting,Management and consulting,0,0,0,1,0,10000,61.0,"10 ,000"
3,Data Scientist,NaN,Role: Data Scientist\nCompany: Arch Systems\nC...,3.9,"Arch Systems, LLC",Remote,Company - Private,Information Technology Support Services,Information Technology,0,0,0,0,0,200,18.0,25
4,Junior Data Scientist,NaN,About Innovecture:\nFounded in 2007 under the ...,4.6,Innovecture,Pune,Company - Private,Information Technology Support Services,Information Technology,0,0,0,0,0,200,17.0,25
